# Seed Type Determination Rating usingSingle Linkage Agglomerative (Bottom-Up) Clustering Technique (STHC-AS)
## Machine Learning Project #3 
## Avi Amalanshu (20EC30063)
We are going to perform k-means clustering on the ```seed.csv``` dataset for various k. The best value of k will be used for agglomerative single-linkage bottom-up clustering. We will compare the clusterings generated by the two methods.


## Setting up some global parameters

In [ ]:
# Roll Number: 20EC30063
# Project Code: STHC-AS
# Project Title: Seed Type Determination Rating usingSingle Linkage Agglomerative (Bottom-Up) Clustering Technique

K_TEST = 3 # Number of clusters for the baseline k-means clustering
ITER_KM = 20 # Number of iterations for the baseline k-means clustering

## Importing some useful (auxiliary) libraries

In [ ]:
import csv
import pandas as pd
import numpy as np
import pprint
import networkx as nx
from networkx.algorithms import minimum_spanning_tree
import scipy.spatial.distance as dist
#from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1BZrusNGN7DWfbqqshRRbGnvXaLHsXKIP/view #seed

Downloading...
From: https://drive.google.com/uc?id=1BZrusNGN7DWfbqqshRRbGnvXaLHsXKIP
To: /content/seeds.csv
100% 9.53k/9.53k [00:00<00:00, 19.3MB/s]


## Importing the dataset

In [ ]:
df = pd.read_csv('seeds.csv')
df.head()

,A,P,C,LK,WK,A_Coef,LKG,target
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0


## Distance Metric: Cosine Similarity
Throughout this experiment, wherever there is a notion of "distance" we will use cosine distance, which is $1-\text{cosine similarity}$ where $$\text{cosine similarity}_{u,v} = \frac{u\cdot v}{\sqrt{(u\cdot u)(v\cdot v)}}$$ From inspection we see that cosine similarity is close to 1 for vectors that "resemble" each other, 0 for those unrelated (orthogonal), and -1 for those which are "adversarial" (antiparallel). And, it is easy to verify that cosine distance is a valid distance metric.

In [ ]:
def cosine_similarity(u, v):
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def distance(u, v):
  return 1 - cosine_similarity(u, v)

## K-means clustering
Arguments:
* Pandas dataframe ```df``` (input data)
* Integer $k$ (number of clusters)
* Integer ```num_iters``` (Number of iterations to wait for convergence

Returns:
* $k$-length np array ```cluster_means``` which are the cluster centroids
* $k$-length list of lists ```clusters``` where each sub-list is the indices of a cluster. 

This method iteratively performs k-means clustering by randomly initializing centroids, clustering based on those, and updating the centroids.

In [ ]:
def k_means(df, k=K_TEST, num_iters=ITER_KM):
  """
  Method to perform k-means clustering on a pandas dataframe.
  Args:
  - df (Pandas dataframe, input data)
  - k (integer, #clusters)
  - num_iters (integer, #iterations for k-means clustering)
  Returns:
  - cluster_means (np.array, ith element is cluster centroids for ith cluster)
  - clusters (list of lists, ith list is points in ith cluster)
  """
  # Random initialization for cluster centroids from dataset
  cluster_means = df.sample(k).values
  for i in range(num_iters):
    # Create an empty list for each cluster
    clusters = [[] for _ in range(k)]
    
    # Assign each data point to the nearest cluster mean
    for j in range(len(df)):
      similarities = [cosine_similarity(df.iloc[j], cluster_means[l]) for l in range(k)]
      nearest_cluster = np.argmax(similarities)
      clusters[nearest_cluster].append(df.iloc[j])

    # Update cluster means to the new centroid
    for j in range(k):
      cluster_means[j] = np.mean(clusters[j], axis=0)
  
  return cluster_means, clusters

## Some helper file I/O code
* ```cluster_save``` writes the latest clustering for a given $k$ to a txt file where each line is a list of points in a certain cluster.
* ```k_means_load``` loads said data to a list of sets. (For kmeans only)

In [ ]:
def cluster_save(clusters, k = K_TEST, name='kmeans'):
  """
  Method that produces a file with k lines. Each line is a comma-separated list of datapoints of a cluster.
  Args:
  - cluster_means (return)
  - clusters (return)
  - k (number of clusters)
  """

  with open(f'{name}_{k}.txt', "w") as f:
    # sorting based on least index in each cluster
    if name == "agglomerative":
      g = range(len(clusters))

      for i in g:
        cluster_data = ",".join([str(sample) for sample in clusters[i]])  # Convert sample.name to str
        f.write(f"{cluster_data}\n")

    else:
      g = {i:x[0].name for i,x in enumerate(clusters)}
      g = {k: v for k, v in sorted(g.items(), key=lambda item: item[1])}

      for i in g:
        cluster_data = ",".join([str(sample.name) for sample in clusters[i]])  # Convert sample.name to str
        f.write(f"{cluster_data}\n")

def k_means_load(k=K_TEST):
    clusters = []
    with open('kmeans_{}.txt'.format(k), 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            data_points = [int(dp) for dp in row]
            cluster = set(data_points)
            clusters.append(cluster)

    return clusters

## Silhouette Coefficient
We wish to evaluate our clustering. Intuitively, we want points in a cluster to be as close to each other as possible and those outside to be as far. One sufficient statistic is the silhouette coefficient-- effectively, any point in the cluster should be closer than any point in any other cluster (viz. the next closest cluster). 

We evaluate the clustering by the average silhouette coefficient over all points in the set.

In [ ]:
def silhouette_coefficient(df, clusters, cluster_means):
  """
  Method to calculate the Silhouette Coefficient for k-means clustering.
  Args: 
  - df (Pandas dataframe, input data)
  - clusters (list of lists, ith list is points in ith cluster)
  - cluster_means (np.array, ith element is cluster centroids for ith cluster)
  Returns:
  - Silhouette Coefficient (float)
    - For each sample,
      - (a-b)/max(a,b)
      - a = The mean distance between a sample and all other points in the same cluster.
      - b = The mean distance between a sample and all other points in the next nearest cluster.
    - Average over all samples.
  """
  n_samples = len(df)
  n_clusters = len(clusters)

  # Compute a
  a = np.zeros(n_samples)
  for i in range(n_clusters):
    for sample in clusters[i]:
      for other in clusters[i]:
        a[sample.name] = a[sample.name] + distance(sample, other)
  # Compute b
  b = np.zeros(n_samples)
  for i in range(n_clusters):
    min = np.inf
    min_idx = None
    # Identify nearest cluster to the ith cluster
    for j in range(n_clusters):
      if j == i:
        continue
      if distance(cluster_means[i], cluster_means[j]) < min:
        min = distance(cluster_means[i], cluster_means[j])
        min_idx = j
    for sample in clusters[i]:
      for other in clusters[min_idx]:
        b[sample.name] = b[sample.name] + distance(sample, other)

  s = np.divide(b-a,np.maximum(b,a))

  # Compute Silhouette Coefficient
  return np.mean(s)


## Generating k-means baseline
These code cells perform k-means clustering for $k=3,4,5,6$ and writes the clustering result of each to an appropriately named file.

In [ ]:
cluster_means, clusters = k_means(df)
cluster_save(clusters)
s = silhouette_coefficient(df, clusters, cluster_means)
print('k = 3, s = {}'.format(s))

k = 3, s = 0.5824521339040021


In [ ]:
max_s = s
best_k = 3
for i in [4, 5, 6]:
  cluster_means, clusters = k_means(df, k = i)
  cluster_save(clusters, k = i)
  s = silhouette_coefficient(df, clusters, cluster_means)
  if s > max_s:
    max_s = s
    best_k = i
  print('k = {}, s = {}'.format(i, s))
print('Greatest value of s = {} for k = {}'.format(max_s, best_k))

k = 4, s = 0.5230638508698913
k = 5, s = 0.563970047251916
k = 6, s = 0.5421660289617972
Greatest value of s = 0.5824521339040021 for k = 3


## Jaccard Coefficient: Similarity between sets
We expect the clusterings to be the same no matter what we do. If they're not, we want a way to be sure they are close together. The Jaccard Coefficient or IoU does that: the intersection of points in the two sets should be large. But, that is also trivially true for a superset (say)-- it is possible the intersection may be irrelevant to one of the sets. So, we also penalize large unions.
$$J(A,B) = \frac{|A\cap B|}{|A\cup B|}$$

In [ ]:
def jaccard_similarity(setA, setB):
    intersection = setA.intersection(setB)
    union = setA.union(setB)
    return len(intersection) / len(union)

## Representing the data as a graph (distance matrix)
We use two methods: one is iterative (```generate_dist_matrix```) and the other (```generate_dist_matrix_2```) uses an inbuilt scipy function for comparison purposes.

In [ ]:
def generate_dist_matrix(df):
  n = len(df)
  dist_matrix = np.zeros((n,n))
  for i in range(n):
    for j in range(i + 1, n):
      d = distance(df.iloc[i], df.iloc[j])
      dist_matrix[i, j] = d
      dist_matrix[j, i] = d  # symmetric
  return dist_matrix

In [ ]:
def generate_dist_matrix_2(df):
  return dist.squareform(dist.pdist(X=df, metric='cosine'))

## Single Linkage Clustering
We perform single linkage clustering. The basic idea is based upon our intuition for "good" clustering-- we try to maximize the distance between clusters by saying "at the very least, these clusters should be $\epsilon$ apart" and then trying to maximize $\epsilon$. 
### Brute Force
We start with assigning each cluster a single point and then merging the clusters between which the minimum distance is the lowest together until we have only $k$ clusters left.
### Kruskal's algorithm
From the cut theorem, it is easy to verify that the distance between two clusters is the edge length between the edge connecting those clusters. So, we can also achieve the same clustering by removing the most expensive $k-1$ edges from the graph.

In [ ]:
def single_linkage_clustering(dist_matrix, k):
  n = dist_matrix.shape[0]

  # Initialize each data point as a cluster
  clusters = [{i} for i in range(n)]

  while len(clusters) > k:
    # Find the two closest clusters
    min_dist = np.inf
    merge_pair = None
    for i in range(len(clusters)):
      for j in range(i+1, len(clusters)):
        # Find the minimum distance between the two clusters
        dist = min([dist_matrix[p1][p2] for p1 in clusters[i] for p2 in clusters[j]])
        if dist < min_dist:
          min_dist = dist
          merge_pair = (i, j)
    # Merge the two closest clusters
    i, j = merge_pair
    clusters[i] = clusters[i].union(clusters[j])
    clusters.pop(j)
  # Return the final clusters
  return clusters

In [ ]:
def single_linkage_clustering_2(dist_matrix, k):
  n = dist_matrix.shape[0]
  G = nx.from_numpy_array(dist_matrix)
  mst = minimum_spanning_tree(G)
  
  # Sort edges by decreasing weight
  sorted_edges = sorted(mst.edges(data=True), key=lambda x: x[2].get('weight'), reverse=True)
  for edge in sorted_edges[:k-1]:
    mst.remove_edge(edge[0],edge[1]) # Remove the k-1 most expensive edges
  
  # Return the connected components (clusters) after removing these edges
  components = list(nx.connected_components(mst))
  return components

## Driver code
We take the clustering from the best $k$ and use it to perform single linkage clustering on both. We expect similar results, especially with the "target" label as a heuristic. 

In [ ]:
# Generate the distance matrices using both methods
dist_matrix = generate_dist_matrix(df)
dist_matrix_2 = generate_dist_matrix_2(df)

# Perform clustering
clusters_KM = k_means_load(best_k) # Load k-means clustering for k = best_k
clusters_AS = single_linkage_clustering(dist_matrix, k = best_k) # Perform single linkage clustering by brute force for k = best_k
clusters_AS_2 = single_linkage_clustering_2(dist_matrix_2, best_k) # Perform single linkage clustering by Kruskal's algorithm for k = best_k

# Save the agglomerative clustering results
cluster_save(clusters_AS_2, best_k, "agglomerative")

# Compute and report pair-wise Jaccard scores
jaccard_scores = [[],[],[]]

# Between k-means and brute force agglomerative
for set_A in clusters_AS:
    # Compute Jaccard similarity between set_A and each set in clusters_KM
    scores = [jaccard_similarity(set_A, set_B) for set_B in clusters_KM]
    # Append the maximum Jaccard similarity score to jaccard_scores
    jaccard_scores[0].append(max(scores))

# Between k-means and greedy agglomerative
for set_A in clusters_AS_2:
    # Compute Jaccard similarity between set_A and each set in clusters_KM
    scores = [jaccard_similarity(set_A, set_B) for set_B in clusters_KM]
    # Append the maximum Jaccard similarity score to jaccard_scores
    jaccard_scores[1].append(max(scores))

# Between brute force and and greedy agglomerative
for set_A in clusters_AS_2:
    # Compute Jaccard similarity between set_A and each set in clusters_AS
    scores = [jaccard_similarity(set_A, set_B) for set_B in clusters_AS]
    # Append the maximum Jaccard similarity score to jaccard_scores
    jaccard_scores[2].append(max(scores))

# Print the Jaccard similarity scores
print('Jaccard scores for set-wise closest clusters in k-means and brute force:\n\t{}'.format(jaccard_scores[0]))
print('Jaccard scores for set-wise closest clusters in k-means and greedy:\n\t{}'.format(jaccard_scores[1]))
print('Jaccard scores for set-wise closest clusters in brute force and greedy:\n\t{}'.format(jaccard_scores[2]))

Jaccard scores for set-wise closest clusters in k-means and brute force:
	[0.5323741007194245, 0.015151515151515152, 0.9154929577464789]
Jaccard scores for set-wise closest clusters in k-means and greedy:
	[0.5323741007194245, 0.015151515151515152, 0.9154929577464789]
Jaccard scores for set-wise closest clusters in brute force and greedy:
	[1.0, 1.0, 1.0]
